In [1]:
import numpy as np
import rioxarray
from xrspatial.zonal import stats
from rasterio.windows import from_bounds
import rasterio
import xarray as xr
from dask.distributed import Client, LocalCluster

In [18]:
cluster = LocalCluster(n_workers=8)
client = Client(cluster)
client.dashboard_link

C:\Users\mweber\AppData\Local\miniforge3\envs\xarray_spatial\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54235 instead
  warnings.warn(


'http://127.0.0.1:54235/status'

In [19]:
LandscapeLayer = 'O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Projects/StreamCat/LandscapeRasters/QAComplete/CanalsDitches.tif'
inZoneData = 'O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Resource/Physical/HYDROLOGY/NHDPlusV21/NHDPlusPN/NHDPlus17/NHDPlusCatchment/cat'

In [20]:
izd_array = rioxarray.open_rasterio(inZoneData, chunks=True).sel(band=1).drop_vars('band')
izd_array

<xarray.DataArray (y: 51025, x: 42353)> Size: 9GB
dask.array<getitem, shape=(51025, 42353), dtype=int32, chunksize=(792, 42353), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 339kB -2.342e+06 -2.342e+06 ... -1.071e+06
  * y            (y) float64 408kB 3.621e+06 3.621e+06 ... 2.091e+06 2.091e+06
    spatial_ref  int64 8B 0
Attributes:
    _FillValue:    -2147483647
    scale_factor:  1.0
    add_offset:    0.0

In [21]:
transform = izd_array.rio.transform()
bounds = izd_array.rio.bounds()
window = from_bounds(*bounds, transform)
window

Window(col_off=np.float64(0.0), row_off=np.float64(0.0), width=np.float64(42353.0), height=np.float64(51024.999999999985))

In [22]:
with rasterio.open(LandscapeLayer) as src:
    ll_array = src.read(1, window=window)
ll_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [23]:
ll_xarray =  xr.DataArray(
        data=ll_array,
        dims=['y', 'x']
    ).chunk(izd_array.chunksizes)
ll_xarray

<xarray.DataArray (y: 51025, x: 42353)> Size: 2GB
dask.array<xarray-<this-array>, shape=(51025, 42353), dtype=uint8, chunksize=(792, 42353), chunktype=numpy.ndarray>
Dimensions without coordinates: y, x

In [24]:
outTable = stats(izd_array, ll_xarray, nodata_values=izd_array.rio.nodata)
outTable

,zone,mean,max,min,sum,std,var,count
npartitions=1,,,,,,,,
,int32,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...


In [25]:
%%time
final_df = outTable.compute()

C:\Users\mweber\AppData\Local\miniforge3\envs\xarray_spatial\Lib\site-packages\distributed\client.py:3361: UserWarning: Sending large graph of size 2.01 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: total: 2min 1s
Wall time: 23min 58s


In [ ]:
final_df.head()

In [ ]:
final_df.to_csv('medium_res_stats_dask_test.csv')